In [ ]:
import pandas as pd
import json
import plotly.express as px
import numpy as np

In [ ]:
microdados_ = pd.read_pickle('microdados_.pkl')
habitos_estudo = pd.read_csv('microdados_enem_2022/DADOS/QUEST_HAB_ESTUDO.csv', encoding='latin1', sep=';')


In [ ]:
microdados_.head()

### 01. Índice de Condição Prévia

In [ ]:
def icp_comp_01(faixa):
    if 'Acima' in faixa:
        val_ref = 5
    elif faixa=='Nenhuma Renda':
        val_ref = 1
    elif '1.818,00' in faixa[-10:]:
        val_ref = 1
    elif '2.424,00' in faixa[-10:]:
        val_ref = 2
    elif '3.030,00' in faixa[-10:]:
        val_ref = 2
    elif '3.636,00' in faixa[-10:]:
        val_ref = 2
    elif '4.848,00' in faixa[-10:]:
        val_ref = 2
    elif '6.060,00' in faixa[-10:]:
        val_ref = 3
    elif '7.272,00' in faixa[-10:]:
        val_ref = 3
    elif '8.484,00' in faixa[-10:]:
        val_ref = 3
    elif '9.696,00' in faixa[-10:]:
        val_ref = 3
    elif '10.908,00' in faixa[-10:]:
        val_ref = 3
    elif '12.120,00' in faixa[-10:]:
        val_ref = 4
    elif '14.544,00' in faixa[-10:]:
        val_ref = 4
    elif '18.180,00' in faixa[-10:]:
        val_ref = 4
    elif '24.240,00' in faixa[-10:]:
        val_ref = 4
    else:
        val_ref = 1
    return val_ref


microdados_['icp_comp_01'] = microdados_['Q006'].apply(lambda x: icp_comp_01(x))

In [ ]:
def icp_comp_02(row):
    # celular
    if row['Q022']=='Não.':
        n_cel = 0
    elif row['Q022']=='Sim, um.':
        n_cel = 1
    elif row['Q022']=='Sim, dois.':
        n_cel = 2
    elif row['Q022']=='Sim, três.':
        n_cel = 3
    else:
        n_cel = 4
    # computador
    if row['Q024']=='Não.':
        n_comp = 0
    elif row['Q024']=='Sim, um.':
        n_comp = 1
    elif row['Q024']=='Sim, dois.':
        n_comp = 2
    elif row['Q024']=='Sim, três.':
        n_comp = 3
    else:
        n_comp = 4 
    # internet
    if row['Q025']=='Não':
        internet = 0
    else:
        internet = 1
    return n_cel*.5 + n_comp + internet*2.5

microdados_['icp_comp_02_parcial'] = microdados_.apply(icp_comp_02, axis=1)



In [ ]:
quintil = list(microdados_['icp_comp_02_parcial'].quantile([.2,.4,.6,.8]))

def ident_quintil(num):
    if num <= quintil[0]:
        y = 1
    elif num <= quintil[1]:
        y = 2
    elif num <= quintil[2]:
        y = 3
    elif num <= quintil[3]:
        y = 4
    else:
        y = 5
    return y

microdados_['icp_comp_02'] = microdados_['icp_comp_02_parcial'].apply(lambda x: ident_quintil(x))

In [ ]:
def icp_comp_03(row):
    # instrucao_pai
    if (row['Q001']=='Nunca estudou.')|((row['Q001']=='Não sei.')) :
        inst_pai = 0
    elif row['Q001']=='Não completou a 4ª série/5º ano do Ensino Fundamental.':
        inst_pai = 1
    elif row['Q001']=='Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.':
        inst_pai = 2
    elif row['Q001']=='Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.':
        inst_pai = 3
    elif row['Q001']=='Completou o Ensino Médio, mas não completou a Faculdade.':
        inst_pai = 4
    elif row['Q001']=='Completou a Faculdade, mas não completou a Pós-graduação.':
        inst_pai = 5
    else:
        inst_pai = 6
    # instrucao_mae
    if (row['Q002']=='Nunca estudou.')|((row['Q002']=='Não sei.')) :
        inst_mae = 0
    elif row['Q002']=='Não completou a 4ª série/5º ano do Ensino Fundamental.':
        inst_mae = 1
    elif row['Q002']=='Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.':
        inst_mae = 2
    elif row['Q002']=='Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.':
        inst_mae = 3
    elif row['Q002']=='Completou o Ensino Médio, mas não completou a Faculdade.':
        inst_mae = 4
    elif row['Q002']=='Completou a Faculdade, mas não completou a Pós-graduação.':
        inst_mae = 5
    else:
        inst_mae = 6 
        
    return inst_pai+inst_mae

microdados_['icp_comp_03_parcial'] = microdados_.apply(icp_comp_03, axis=1)

quintil = list(microdados_['icp_comp_03_parcial'].quantile([.2,.4,.6,.8]))

microdados_['icp_comp_03'] = microdados_['icp_comp_03_parcial'].apply(lambda x: ident_quintil(x))

In [ ]:
microdados_['icp'] = microdados_['icp_comp_01'] + microdados_['icp_comp_02'] + microdados_['icp_comp_03']

In [ ]:
microdados_[['icp', 'nota_media_geral']].groupby(['icp']).mean()

In [ ]:
num=2

print(f"Identificador: {microdados_.iloc[num]['NU_INSCRICAO']}")
print(f"Faixa de Renda: {microdados_.iloc[num]['Q006']}, o que o coloca com índice {microdados_.iloc[num]['icp_comp_01']} no primeiro componente, numa escala de 1 a 5")
print(f"Celulares na residência: {microdados_.iloc[num]['Q022']}")
print(f"Computadores na residência: {microdados_.iloc[num]['Q024']}")
print(f"Internet na residência: {microdados_.iloc[num]['Q025']}, o que o coloca com índice {microdados_.iloc[num]['icp_comp_02']} no segundo componente, numa escala de 1 a 5")
print(f"Formação do pai: {microdados_.iloc[num]['Q001']}")
print(f"Formação da mãe: {microdados_.iloc[num]['Q002']}, o que o coloca com índice {microdados_.iloc[num]['icp_comp_03']} no terceiro componente, numa escala de 1 a 5")
print(f"Índice de Condição Prévia: {microdados_.iloc[num]['icp']}, numa escala de 1 a 15")

In [ ]:
microdados_.columns

### 02. ICP por regiões

In [ ]:
# Carregando Json com metadados de mapa de municípios
objeto_geo_cidades = open('geojs-100-mun.json',)
geo_cidades = json.load(objeto_geo_cidades)

In [ ]:
microdados_geo_mun = microdados_[pd.isna(microdados_['CO_MUNICIPIO_ESC'])==False][['CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'SG_UF_ESC', 'nota_media_geral', 'nota_media_objetivas', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'icp']]
microdados_geo_mun['id'] = microdados_geo_mun['CO_MUNICIPIO_ESC'].apply(lambda x: str(int(x)))
microdados_geo_mun_agg = microdados_geo_mun[['id', 'icp']].groupby(['id']).mean().reset_index()

microdados_geo_mun_pub = microdados_[(pd.isna(microdados_['CO_MUNICIPIO_ESC'])==False)&(microdados_['TP_DEPENDENCIA_ADM_ESC'].isin(['Estadual', 'Municipal']))][['CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'SG_UF_ESC', 'nota_media_geral', 'nota_media_objetivas', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'icp']]
microdados_geo_mun_pub['id'] = microdados_geo_mun_pub['CO_MUNICIPIO_ESC'].apply(lambda x: str(int(x)))
microdados_geo_mun_pub_agg = microdados_geo_mun_pub[['id', 'icp']].groupby(['id']).mean().reset_index()


In [ ]:
print(microdados_geo_mun_agg['icp'].min(), 
      microdados_geo_mun_agg['icp'].max(), 
      microdados_geo_mun_pub_agg['icp'].min(),
      microdados_geo_mun_pub_agg['icp'].max())

In [ ]:
# Mapa: Municípios, icp
mapa_icp = px.choropleth_mapbox(
    data_frame = microdados_geo_mun_agg, 
    geojson = geo_cidades, 
    locations='id', 
    featureidkey='properties.id',
    color='icp',
    #color_continuous_scale= 'reds',
    range_color=(3, 13),
    mapbox_style='open-street-map',
    zoom=2.5, 
    center = {"lat": -17.14, "lon": -57.33},
    opacity=1,
    labels={'icp':'Índice de Condição Prévia dos Alunos', 
            'id' : 'Código do município'
    },
    width = 1000,
    height = 800,
    title = 'ICP médio, por municípios'
)

# Ajustando as margens
mapa_icp.update_layout(margin={'r':0,'t':0,'l':0, 'b':0})

# Reduzindo a largura das bordas dos municípios
mapa_icp.update_traces(marker_line_width=0.01)

# Exibindo o mapa
mapa_icp.show()

In [ ]:
# Mapa: Municípios, icp_publicas
mapa_icp = px.choropleth_mapbox(
    data_frame = microdados_geo_mun_pub_agg, 
    geojson = geo_cidades, 
    locations='id', 
    featureidkey='properties.id',
    color='icp',
    #color_continuous_scale= 'reds',
    range_color=(3, 13),
    mapbox_style='open-street-map',
    zoom=2.5, 
    center = {"lat": -17.14, "lon": -57.33},
    opacity=1,
    labels={'icp':'Índice de Condição Prévia dos Alunos', 
            'id' : 'Código do município'
    },
    width = 1000,
    height = 800,
    title = 'ICP médio, por municípios - escolas públicas'
)

# Ajustando as margens
mapa_icp.update_layout(margin={'r':0,'t':0,'l':0, 'b':0})

# Reduzindo a largura das bordas dos municípios
mapa_icp.update_traces(marker_line_width=0.01)

# Exibindo o mapa
mapa_icp.show()

In [ ]:
dicio_regioes = {'SE': 'Nordeste',
                 'AL': 'Nordeste',
                 'PE': 'Nordeste',
                 'PB': 'Nordeste',
                 'RN': 'Nordeste',
                 'CE': 'Nordeste',
                 'PI': 'Nordeste',
                 'MA': 'Nordeste',
                 'BA': 'Nordeste',
                 'AP': 'Norte',
                 'AM': 'Norte',
                 'AC': 'Norte',
                 'PA': 'Norte',
                 'RR': 'Norte',
                 'RO': 'Norte',
                 'TO': 'Norte',
                 'MG': 'Sudeste',
                 'SP': 'Sudeste',
                 'RJ': 'Sudeste',
                 'ES': 'Sudeste',
                 'RS': 'Sul',
                 'SC': 'Sul',
                 'PR': 'Sul',
                 'DF': 'Centro-Oeste',
                 'MT': 'Centro-Oeste',
                 'MS': 'Centro-Oeste',
                 'GO': 'Centro-Oeste'}

def regiao(uf):
    if uf!=uf:
        regiao = 'Sem informação'
    else:
        regiao = dicio_regioes[uf]
    return regiao

microdados_['regiao'] = microdados_['SG_UF_ESC'].map(regiao)
microdados_geo_mun_pub['regiao'] = microdados_geo_mun_pub['SG_UF_ESC'].map(regiao)


In [ ]:
microdados_[microdados_['regiao']!='Sem informação'][['regiao', 'icp']].groupby(['regiao']).mean().sort_values('icp', ascending=False)


In [ ]:
microdados_geo_mun_pub[['regiao', 'icp']].groupby(['regiao']).mean().sort_values('icp', ascending=False)


In [ ]:
microdados_[microdados_['regiao']!='Sem informação'][['regiao', 'nota_media_geral']].groupby(['regiao']).mean().sort_values('nota_media_geral', ascending=False)


In [ ]:
microdados_geo_mun_pub[['regiao', 'nota_media_geral']].groupby(['regiao']).mean().sort_values('nota_media_geral', ascending=False)


### 03. Grupos

In [ ]:
microdados_[['TP_COR_RACA', 'icp', 'nota_media_geral']].groupby(['TP_COR_RACA']).mean().sort_values('icp', ascending=False)


In [ ]:
microdados_[['TP_SEXO', 'icp', 'nota_media_geral']].groupby(['TP_SEXO']).mean().sort_values('icp', ascending=False)


In [ ]:
microdados_[['icp', 'TP_COR_RACA', 'nota_media_geral']].groupby(['icp', 'TP_COR_RACA']).mean().sort_values(['icp', 'nota_media_geral'], ascending=[True, False]).reset_index().to_excel('icp_raca_nota.xlsx')


In [ ]:
microdados_[microdados_['TP_DEPENDENCIA_ADM_ESC'].isin(['Estadual', 'Municipal'])][['icp', 'TP_COR_RACA', 'nota_media_geral']].groupby(['icp', 'TP_COR_RACA']).mean().sort_values(['icp', 'nota_media_geral'], ascending=[True, False]).reset_index()#.to_excel('icp_raca_nota_pub.xlsx')


In [ ]:
df_raca_b_p = microdados_[(microdados_['TP_DEPENDENCIA_ADM_ESC'].isin(['Estadual', 'Municipal'])&(microdados_['TP_COR_RACA']=='Branca'))][['icp', 'nota_media_geral']].groupby(['icp']).mean().sort_values(['icp', 'nota_media_geral'], ascending=[True, False]).reset_index().merge(
microdados_[(microdados_['TP_DEPENDENCIA_ADM_ESC'].isin(['Estadual', 'Municipal'])&(microdados_['TP_COR_RACA']=='Preta'))][['icp', 'nota_media_geral']].groupby(['icp']).mean().sort_values(['icp', 'nota_media_geral'], ascending=[True, False]).reset_index(), on = 'icp', how = 'inner')

df_raca_b_p['dif'] = df_raca_b_p['nota_media_geral_x'] - df_raca_b_p['nota_media_geral_y']
df_raca_b_p['icp_'] = df_raca_b_p['icp'].apply(lambda x: 'ICP ' + str(x))
df_raca_b_p[['icp_', 'dif']]


In [ ]:
microdados_.to_pickle('microdados_.pkl')